In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline
import matplotlib
font = {'family' : 'sans-serif',
        'size'   : 14}

matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 

In [5]:
# ---------------------------------------
# WARM ELECTRONS
# ---------------------------------------
class warm_e_mtsi:

    sqrt_pi = np.sqrt(np.pi)
    sqrt_2 = np.sqrt(2.0)
    
    def __init__(self,U0,vthe,nspi,vthi,ni_n0,qi_qe,wpe_wce,mi_me,theta):

        # number of bessel functions to include in the sims
        self.Nbessel = 10

        # number of ion species
        self.nspi = nspi
                
        # U0
        self.U0 = U0

        # qi
        self.qi_qe = qi_qe

        # ni
        self.ni_n0 = ni_n0
        
        # Wce
        self.Wce = -1/wpe_wce

        # vthi and vthe
        self.vthi = self.sqrt_2*vthi
        self.vthe = self.sqrt_2*vthe

        self.Te = vthe**2
        self.Ti = mi_me*vthi**2
        
        self.mi_me = mi_me

        self.cost = np.cos(theta)
        self.sint = np.sin(theta)        
        
    # ---------------------------
    # plasma dispersion function
    # ---------------------------
    def Z(self, xi):    
        Z_ = 1j*self.sqrt_pi*wofz(xi)
        return Z_


    # ---------------------------
    # computes the sum over bessel
    # functions
    # input : frequency w, parameter lambda
    # ---------------------------
    def Isum( self, w, lambda_ ):
        
        Nb = self.Nbessel   # number of functions to include
        
        m = np.arange(-Nb, Nb + 1)
        kpar  = self.k_*self.cost

        
#        xi = (w - m*self.Wce)/(kpar*self.vthe*self.sqrt_2)        
#        tmp_ = ive(m, lambda_)*self.Z(xi)
#        res_ = np.sum(tmp_)

        res_ = 0
        for m_ in m:
            xi = (w - m_*self.Wce)/(kpar*self.vthe)          
            res_ += ive(m_, lambda_)*self.Z(xi)

        return res_
    
    # ---------------------------
    # determinant of the mode coupling
    # ---------------------------
    def determinant( self, w_2D ):
        w = w_2D[0] + 1j*w_2D[1]
        k  = self.k_
        kpar = k*self.cost
        kper = k*self.sint
        lambda_ = kper**2*self.Te/self.Wce**2

        esum_ = self.Isum(w,lambda_)*w/(self.vthe*kpar)
        el_response = 1.0/self.Te*(1+esum_)

        res_ =  - el_response - k**2

        for sp in np.arange(self.nspi):
            
            xi = (w - kper*self.U0[sp])/(k*self.vthi[sp])
            Z_   =  self.Z(xi)
            Zprime = -2.0*(1.0+xi*Z_)

            res_ += self.ni_n0[sp]*self.qi_qe[sp]**2*Zprime/self.mi_me[sp]/self.vthi[sp]**2 
        return (res_.real,res_.imag)/(k**2)
                
        
    # ---------------------------------------
    # Nonlinear solver: for a given k, return
    # the solution w
    # inputs : value of k and guess wg 
    # ---------------------------------------
    def solve(self,k,wg):

        # store the value of k, so that it's accessible inside the function called by the solver
        self.k_ = k

        # initial guess
        wg_ = [wg.real,wg.imag]
        sol = root(self.determinant, wg_ )
        res_ = sol.x[0] + 1j*sol.x[1]
        
        return (res_,sol.success)
        